In [1]:
from cpt import * 

# Conditional Probability Tables

In [2]:
V_burglar = Variable('burglar', [True, False])
V_storm = Variable('storm', [True, False])
V_report = Variable('report', [True, False])
V_window = Variable('window', [True, False])
V_alarm = Variable('alarm', [True, False])

P_burglar = CPT([0.001, 0.999], [V_burglar])
P_storm = CPT([0.01, 0.99], [V_storm])

# P(report | storm)
CP_report = CPT([0.9, 0.02, 0.1, 0.98], [V_report, V_storm])

# P(window | storm, burglar)
CP_window = CPT([0.95, 0.9, 0.5, 0.0, 0.05, 0.1, 0.5, 1.0],
                [V_window, V_burglar, V_storm])

# P(alarm | window)
CP_alarm = CPT([0.95, 0.01, 0.05, 0.99], [V_alarm, V_window])

In [3]:
# Print some tables
print(P_burglar)

(burglar) value
--------- -----
(True,)   0.001
(False,)  0.999


In [4]:
# Join: Compute P(report|storm), P(storm) => P(storm, report)
P_SR = join(CP_report, P_storm)
print(P_SR)

(report, storm) value
--------------- ------
(True, True)    0.009
(True, False)   0.0198
(False, True)   0.001
(False, False)  0.9702


In [5]:
# Marginalize: Compute P(report) from P(storm, report)
P_R = marginalize(P_SR, ['report'])
print(P_R)

(report) value
-------- ------
(True,)  0.0288
(False,) 0.9712


In [6]:
# Compute the joint distribution over all variables
P_DSRAW = reduce(join, [P_burglar, P_storm, CP_report, CP_window, CP_alarm])
print(P_DSRAW)

(burglar, storm, report, window, alarm) value
--------------------------------------- -----------
(True, True, True, True, True)          8.1225e-06
(True, True, True, True, False)         4.275e-07
(True, True, True, False, True)         4.5e-09
(True, True, True, False, False)        4.455e-07
(True, True, False, True, True)         9.025e-07
(True, True, False, True, False)        4.75e-08
(True, True, False, False, True)        5e-10
(True, True, False, False, False)       4.95e-08
(True, False, True, True, True)         1.6929e-05
(True, False, True, True, False)        8.91e-07
(True, False, True, False, True)        1.98e-08
(True, False, True, False, False)       1.9602e-06
(True, False, False, True, True)        0.000829521
(True, False, False, True, False)       4.3659e-05
(True, False, False, False, True)       9.702e-07
(True, False, False, False, False)      9.60498e-05
(False, True, True, True, True)         0.004270725
(False, True, True, True, False)        0.000224775


In [7]:
# Add observations alarm=True and report=False
P_posteriori = eliminate(eliminate(P_DSRAW, 'alarm', True), 'report', False)
# Marginalize to burglar
P_burglar_post = marginalize(P_posteriori, 'burglar')
print(P_burglar_post)

(burglar) value
--------- ---------------
(True,)   0.0755592262412
(False,)  0.924440773759


# Factor Graph

In [8]:
f = FactorGraph([V_burglar, V_storm, V_window, V_alarm, V_report], 
                [P_burglar, P_storm, CP_window, CP_alarm, CP_report])

In [9]:
f.forward_backward_pass()

In [10]:
f.posterior(V_alarm)

(alarm)  value
-------- ----------
(True,)  0.01554177
(False,) 0.98445823

### Now build a factor graph with our additional observations

In [11]:
NEW_P_alarm = CPT([1.0, 0.0], [V_alarm])
NEW_P_report = CPT([0.0, 1.0], [V_report])

f2 = FactorGraph([V_burglar, V_storm, V_window, V_alarm, V_report], 
                 [P_burglar, P_storm, CP_window, CP_alarm, CP_report,
                  NEW_P_alarm, NEW_P_report])

f2.forward_backward_pass()
print(f2.posterior(V_burglar))

(burglar) value
--------- ---------------
(True,)   0.0755592262412
(False,)  0.924440773759
